In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame 
from datetime import datetime

In [85]:
def convertDateTimeTupleIntoDateTimeObject(dateTimeTuple):
    
    dateString = dateTimeTuple[0]
    timeString = dateTimeTuple[1]
    
    dateObject = datetime.strptime(dateString, "%m/%d/%Y").date()
    timeObject = None
    
    if timeString == "24:00":
        timeString = "23:00"
        timeObject = datetime.strptime(timeString, "%H:%M").time()
        
    else: 
        timeObject = datetime.strptime(timeString, "%H:%M").time()
        timeObject = timeObject.replace(hour=timeObject.hour-1)

    return str(datetime.combine(dateObject, timeObject))
    
# Reading load data - times binned by hour
load_df = pd.read_csv("./system_load_by_region/cdr.00013101.0000000000000000.20140102.055001.ACTUALSYSLOADWZNP6345.csv")
load_df = load_df.rename(columns={"OperDay": "Date"}).drop(["TOTAL", "DSTFlag"],axis=1)

dates = load_df["Date"].tolist()
hours = load_df["HourEnding"].tolist()
datesHours = zip(dates, hours)

dateTimes = map(convertDateTimeTupleIntoDateTimeObject, datesHours)
load_df["Date"] = dateTimes
load_df = load_df.drop("HourEnding",axis=1)

# load_df.head()
load_df

,Date,COAST,EAST,FAR_WEST,NORTH,NORTH_C,SOUTHERN,SOUTH_C,WEST
0,2014-01-01 00:00:00,8913.01,1491.29,1783.56,968.27,11923.53,3230.89,6191.00,1312.92
1,2014-01-01 01:00:00,8718.73,1471.86,1777.56,957.07,11923.27,3242.57,6160.06,1314.45
2,2014-01-01 02:00:00,8554.86,1457.77,1783.28,961.51,11913.34,3207.31,6133.25,1299.35
3,2014-01-01 03:00:00,8444.40,1469.57,1780.23,979.46,11984.70,3150.04,6090.27,1308.81
4,2014-01-01 04:00:00,8436.14,1498.56,1784.08,993.57,12250.34,3114.54,6129.80,1344.78
5,2014-01-01 05:00:00,8570.15,1543.28,1798.99,1024.36,12678.14,3117.14,6264.58,1364.00
6,2014-01-01 06:00:00,8719.81,1580.37,1806.58,1055.84,13023.76,3171.56,6433.34,1406.47
7,2014-01-01 07:00:00,8665.54,1588.44,1819.77,1074.47,13154.07,3155.70,6492.82,1443.59
8,2014-01-01 08:00:00,8661.20,1573.47,1818.99,1064.53,12956.24,3159.13,6427.81,1425.48
9,2014-01-01 09:00:00,8778.05,1531.44,1784.43,1048.56,12628.14,3200.12,6206.40,1366.63


In [ ]:
def roundUTCHourUp(dateString):
    dateObject = datetime.strptime(dateString, "%Y-%m-%d %H:%M:%S")
    newHour = (dateObject.hour + 1) % 24
    dateObject = dateObject.replace(hour=newHour)
    return dateObject.strftime("%Y-%m-%d %H:00:00")

def reformatUTCToDate(dateString): 
    return str(
        datetime.strptime(dateString, "%Y-%m-%d %H:%M:%S").strftime("%m/%d/%Y"))

def setOrderWeatherDF(df): 
    return df[["Date","TemperatureF"]]


# Reading weather data - round up timestamps to the nearest hour

weather_df1 = pd.read_csv("./weather_data/KDAL_20140101.csv")
weather_df1 = weather_df1[["DateUTC", "TemperatureF"]]

weather_df1["Date"] = weather_df1["DateUTC"].apply(roundUTCHourUp)
weather_df1 = setOrderWeatherDF(weather_df1)


# weather_df1["DateUTC"] = weather_df1["DateUTC"].apply(lambda dt: )
weather_df1

In [15]:
weather_df2 = pd.read_csv("./weather_data/KDAL_20140102.csv")
weather_df2 = weather_df2[["DateUTC", "TemperatureF"]]
weather_df2["Date"] = weather_df2["DateUTC"].apply(reformatUTCToDate)
weather_df2["HourEnding"] = weather_df2["DateUTC"].apply(roundUTCHourUp)
weather_df2 = weather_df2.drop("DateUTC",axis=1)
weather_df2 = setOrderWeatherDF(weather_df2)


In [78]:
# Combine All Dallas DataFrames
weather_df3 = weather_df1.append(weather_df2)
# weather_df3.head()

In [87]:
load_df

,Date,COAST,EAST,FAR_WEST,NORTH,NORTH_C,SOUTHERN,SOUTH_C,WEST
0,2014-01-01 00:00:00,8913.01,1491.29,1783.56,968.27,11923.53,3230.89,6191.00,1312.92
1,2014-01-01 01:00:00,8718.73,1471.86,1777.56,957.07,11923.27,3242.57,6160.06,1314.45
2,2014-01-01 02:00:00,8554.86,1457.77,1783.28,961.51,11913.34,3207.31,6133.25,1299.35
3,2014-01-01 03:00:00,8444.40,1469.57,1780.23,979.46,11984.70,3150.04,6090.27,1308.81
4,2014-01-01 04:00:00,8436.14,1498.56,1784.08,993.57,12250.34,3114.54,6129.80,1344.78
5,2014-01-01 05:00:00,8570.15,1543.28,1798.99,1024.36,12678.14,3117.14,6264.58,1364.00
6,2014-01-01 06:00:00,8719.81,1580.37,1806.58,1055.84,13023.76,3171.56,6433.34,1406.47
7,2014-01-01 07:00:00,8665.54,1588.44,1819.77,1074.47,13154.07,3155.70,6492.82,1443.59
8,2014-01-01 08:00:00,8661.20,1573.47,1818.99,1064.53,12956.24,3159.13,6427.81,1425.48
9,2014-01-01 09:00:00,8778.05,1531.44,1784.43,1048.56,12628.14,3200.12,6206.40,1366.63


In [88]:
weather_df1

,Date,TemperatureF
0,2014-01-01 07:00:00,39.9
1,2014-01-01 08:00:00,39.0
2,2014-01-01 09:00:00,37.9
3,2014-01-01 10:00:00,37.9
4,2014-01-01 11:00:00,37.0
5,2014-01-01 12:00:00,36.0
6,2014-01-01 13:00:00,36.0
7,2014-01-01 14:00:00,37.0
8,2014-01-01 15:00:00,39.9
9,2014-01-01 16:00:00,45.0


In [86]:
# Merge the aggregate of the weather data frames with the aggregate of the load frames
final = pd.merge(load_df, weather_df1, on="Date")
final

,Date,COAST,EAST,FAR_WEST,NORTH,NORTH_C,SOUTHERN,SOUTH_C,WEST,TemperatureF
0,2014-01-01 00:00:00,8913.01,1491.29,1783.56,968.27,11923.53,3230.89,6191.00,1312.92,68.0
1,2014-01-01 07:00:00,8665.54,1588.44,1819.77,1074.47,13154.07,3155.70,6492.82,1443.59,39.9
2,2014-01-01 08:00:00,8661.20,1573.47,1818.99,1064.53,12956.24,3159.13,6427.81,1425.48,39.0
3,2014-01-01 09:00:00,8778.05,1531.44,1784.43,1048.56,12628.14,3200.12,6206.40,1366.63,37.9
4,2014-01-01 10:00:00,8759.46,1459.10,1774.87,999.31,12160.81,3201.32,6065.07,1297.79,37.9
5,2014-01-01 11:00:00,8661.44,1366.90,1765.61,954.26,11608.01,3192.11,5872.33,1248.21,37.0
6,2014-01-01 12:00:00,8595.00,1274.72,1732.35,901.68,11001.45,3117.07,5683.01,1191.92,36.0
7,2014-01-01 13:00:00,8486.33,1202.35,1692.05,859.03,10404.83,3040.75,5464.22,1113.09,36.0
8,2014-01-01 14:00:00,8413.84,1141.63,1665.00,820.15,9909.38,2969.03,5301.20,1076.58,37.0
9,2014-01-01 15:00:00,8391.99,1129.52,1648.70,806.77,9639.97,2916.82,5250.64,1042.28,39.9


In [99]:
import os
filter(lambda x: x[-4:] == ".csv", os.listdir('system_load_by_region'))

['cdr.00013101.0000000000000000.20140101.055000.ACTUALSYSLOADWZNP6345.csv',
 'cdr.00013101.0000000000000000.20140102.055001.ACTUALSYSLOADWZNP6345.csv',
 'cdr.00013101.0000000000000000.20140103.055000.ACTUALSYSLOADWZNP6345.csv',
 'cdr.00013101.0000000000000000.20140104.055000.ACTUALSYSLOADWZNP6345.csv',
 'cdr.00013101.0000000000000000.20140105.055000.ACTUALSYSLOADWZNP6345.csv',
 'cdr.00013101.0000000000000000.20140106.055000.ACTUALSYSLOADWZNP6345.csv',
 'cdr.00013101.0000000000000000.20140107.055000.ACTUALSYSLOADWZNP6345.csv',
 'cdr.00013101.0000000000000000.20140108.055000.ACTUALSYSLOADWZNP6345.csv',
 'cdr.00013101.0000000000000000.20140109.055000.ACTUALSYSLOADWZNP6345.csv',
 'cdr.00013101.0000000000000000.20140110.055000.ACTUALSYSLOADWZNP6345.csv',
 'cdr.00013101.0000000000000000.20140111.055000.ACTUALSYSLOADWZNP6345.csv',
 'cdr.00013101.0000000000000000.20140112.055000.ACTUALSYSLOADWZNP6345.csv',
 'cdr.00013101.0000000000000000.20140113.055001.ACTUALSYSLOADWZNP6345.csv',
 'cdr.000131

NameError: name 'DataFrame' is not defined